# Import dependencies

In [2]:
# !pip install transformers[sentencepiece]
# !pip install datasets
# !pip install fugashi ipadic
# !pip install unidic_lite
# !apt-get install git-lfs

# Data Preprocess

In [3]:
import json
import glob
def read_json_file(file_path):
    with open(file_path) as f:
        data = json.load(f)
    return data

In [4]:
bn_train_data_path = "./bn_dataset/bn/train"
bn_val_data_path = "./bn_dataset/bn/val"
bn_train_files = glob.glob(bn_train_data_path+"/*.json")
bn_val_files = glob.glob(bn_val_data_path+"/*.json")

In [5]:
bn_val_files

['./bn_dataset/bn/val/validation.json']

In [6]:
from itertools import chain

In [7]:
bn_train_files

['./bn_dataset/bn/train/train.json']

In [8]:
bn_val_files

['./bn_dataset/bn/val/validation.json']

In [9]:
bn_train_data_list= list(map(read_json_file, bn_train_files))
bn_val_data_list= list(map(read_json_file, bn_val_files))

In [10]:
bn_train_list = [i["data"] for i in bn_train_data_list]
bn_val_list = [i["data"] for i in bn_val_data_list]


In [11]:
bn_train_data = list(chain(* bn_train_list))

In [12]:
len(bn_train_data)

1252

In [13]:
len(bn_train_data)

1252

In [14]:
bn_val_data = list(chain(* bn_val_list))

In [15]:
len(bn_val_data)

1252

In [16]:
import random
all_data = bn_train_data + bn_val_data

In [17]:
random.shuffle(all_data)

In [18]:
len(all_data)

2504

In [19]:
percentage = int((len(all_data)*90)/100)
print(percentage)

2253


In [20]:
bn_train_data = all_data[:percentage]
bn_val_data = all_data[percentage:]

In [21]:
from pprint import pprint
for i in bn_train_data[:1]:
    pprint(i)

{'paragraphs': [{'context': '১। গোলআলু Solanum tuberosum   ২। পেঁয়াজ Allium '
                            'cepa   ৩। ধান Oryza sativa   ৪। জবা Hibiscus '
                            'rosa-sinensis   ৫। পাট Corchorus capsularis   ৬। '
                            'আম Mangifera indica   ৭। কাঁঠাল Artocarpus '
                            'heterophyllus   ৮। শাপলা Nymphaea nouchali   ৯। '
                            'রুই মাছ Labeo rohita   ১০। কাতলা Catla catla   '
                            '১১। সিংহ Panthera leo   ১২। রয়েল বেঙ্গল টাইগার '
                            'Panthera tigris   ১৩। ম্যালেরিয়া জীবাণু '
                            'Plasmodium vivax   ১৪। আরশোলা Periplaneta '
                            'americana   ১৫। মৌমাছি Apis indica   ১৬। ইলিশ '
                            'Tenualosa ilisha   ১৭। কুনোব্যাঙ '
                            'Bufo/Duttaphrynus melanostictus   ১৮। দোয়েল '
                            'Copsychus saularis   ১৯। মানুষ Homo sapiens   ২০। '
            

# Model And Tokenizer Initialization

In [22]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer")
tokenizer = AutoTokenizer.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer", use_fast=True)
# tokenizer.do_lower_case = True 

In [23]:
text = 'রাগবি লিগটি বেশিরভাগ নিউজিল্যান্ড এবং অস্ট্রেলিয়ায় বসবাসকারী সামোয়ানরা খেলে, প্রশংসাপত্র প্রয়োজন এনআরএল, সুপার লিগ এবং ঘরোয়া '

string_tokenized = tokenizer.encode(text)
print(string_tokenized)

[0, 266, 264, 1123, 268, 339, 268, 1612, 268, 279, 265, 292, 268, 666, 264, 298, 297, 268, 897, 268, 272, 267, 277, 264, 269, 267, 323, 402, 312, 514, 267, 286, 267, 266, 265, 272, 268, 277, 367, 277, 309, 1431, 264, 535, 264, 266, 289, 281, 264, 274, 288, 277, 367, 798, 264, 345, 265, 272, 379, 285, 267, 453, 312, 280, 264, 390, 267, 266, 285, 267, 674, 832, 358, 735, 8156, 16, 281, 276, 287, 264, 266, 339, 268, 298, 402, 312, 793, 288, 277, 367, 225, 2]


In [24]:
def data_preprocessing(dataset):
    contexts, questions, answers = [], [], []
    for group in dataset:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

In [25]:
train_contexts, train_questions, train_answers = data_preprocessing(bn_train_data)
test_contexts, test_questions, test_answers = data_preprocessing(bn_val_data)

print(len(train_answers))
squad_train = {'answers': train_answers,'context': train_contexts, 'question': train_questions}
squad_test = {'answers': test_answers,'context': test_contexts, 'question': test_questions}

2396


In [27]:
squad_train["answers"][:10]

[{'answer_start': 585, 'text': 'কলেরা জীবাণু'},
 {'answer_start': 202,
  'text': 'যুক্তরাষ্ট্রের ওয়াশিংটন অঙ্গরাজ্যের রেডমন্ড শহরে'},
 {'answer_start': 93, 'text': 'উরুগুয়ে'},
 {'answer_start': 22, 'text': 'তেঁতলে বিলাস'},
 {'answer_start': 404, 'text': 'ফিলিপাইনে'},
 {'answer_start': 604, 'text': 'আবু ধাবি'},
 {'answer_start': 20, 'text': 'আরবী মূলক'},
 {'answer_start': 528, 'text': 'সূর্য সেন'},
 {'answer_start': 352, 'text': 'হায়দ্রাবাদ'},
 {'answer_start': 433, 'text': '১৯৭১'}]

In [28]:
squad_train["question"][:10]

['মানুষের বৈজ্ঞানিক নাম কী ?',
 'মাইক্রোসফট কোম্পানীর সদর অফিস কোথায় ?',
 'সর্বপ্রথম ফুটবল বিশ্বকাপ কোথায় অনুষ্ঠিত হয়েছিল ?',
 'সমরেশ বসু রচিত গঙ্গা বাংলা উপন্যাসের নায়ক কে ?',
 'এশীয় উন্নয়ন ব্যাংকটির সদর দপ্তর কোথায় অবস্থিত ?',
 'সংযুক্ত আরব আমিরাতের রাজধানী কোথায় ?',
 'কাগজ শব্দটি কোন ধরণের বাংলা শব্দ ?',
 'গৌড়ের সর্বশেষ রাজা কে ছিলেন ?',
 'ভারতের অন্ধ্রপ্রদেশ রাজ্যের রাজধানী কোথায় ?',
 'মুহাম্মদ আতাউল গণি ওসমানী কোন সালে বাংলাদেশের সেনাবাহিনীর প্রধান সেনাপতির পদে নিযুক্ত হন ?']

In [29]:
squad_train["context"][:10]

['১। গোলআলু Solanum tuberosum   ২। পেঁয়াজ Allium cepa   ৩। ধান Oryza sativa   ৪। জবা Hibiscus rosa-sinensis   ৫। পাট Corchorus capsularis   ৬। আম Mangifera indica   ৭। কাঁঠাল Artocarpus heterophyllus   ৮। শাপলা Nymphaea nouchali   ৯। রুই মাছ Labeo rohita   ১০। কাতলা Catla catla   ১১। সিংহ Panthera leo   ১২। রয়েল বেঙ্গল টাইগার Panthera tigris   ১৩। ম্যালেরিয়া জীবাণু Plasmodium vivax   ১৪। আরশোলা Periplaneta americana   ১৫। মৌমাছি Apis indica   ১৬। ইলিশ Tenualosa ilisha   ১৭। কুনোব্যাঙ Bufo/Duttaphrynus melanostictus   ১৮। দোয়েল Copsychus saularis   ১৯। মানুষ Homo sapiens   ২০। কলেরা জীবাণু Vibrio cholerae   ২১। গম Triticum aestivum   ২২। ভুট্টা Zea mays   ২৩। মসুর Lens culinaris   ২৪। ছোলা Cicer arietinum   ২৫। মোটর Pisum sativum   ২৬। সোনামুগ Vigna radiate   ২৭। মাসকলাই Vigna mungo   ২৮। খেসারী Lathyrus sativus   ২৯। সয়াবিন Glycine max   ৩০। তিল Sesamum indicum',
 'মাইক্রোসফট কর্পোরেশন (NASDAQ:) যুক্তরাষ্ট্র ভিত্তিক একটি কম্পিউটার প্রযুক্তি প্রতিষ্ঠান। এটি বিভিন্ন কম্পিউটার ডিভাইস

In [30]:
def preprocess_function(examples, tokenizer):
#     print(examples.keys())
#     exit()
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
#         print(answer)
        start_char = answer["answer_start"]
        end_char = answer["answer_start"] + len(answer["text"])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [31]:
tokenized_squad_train = preprocess_function(squad_train,tokenizer)
tokenized_squad_test = preprocess_function(squad_test,tokenizer)

In [32]:
from functools import partial
def generated_dict(tokenized_squad):
    tokenized_squad_data = list(map(lambda x: {
            "input_ids": x[0], 
            "attention_mask": x[1], 
            "start_positions": x[2], 
            "end_positions": x[3]
        },
        zip(
            tokenized_squad["input_ids"], 
            tokenized_squad["attention_mask"],
            tokenized_squad["start_positions"], 
            tokenized_squad["end_positions"]
        )
    ))
    return tokenized_squad_data


In [33]:
add_part = partial(generated_dict)
tokenized_squad_train_new = add_part(tokenized_squad_train)
tokenized_squad_val_new = add_part(tokenized_squad_test)

In [34]:
for i in tokenized_squad_train_new[:1]:
    print(i)

{'input_ids': [0, 274, 264, 269, 276, 301, 265, 266, 279, 392, 291, 267, 370, 264, 269, 268, 270, 297, 264, 274, 278, 289, 4685, 2, 2, 502, 283, 313, 288, 272, 1320, 276, 225, 55, 2163, 831, 2145, 557, 89, 3698, 2481, 2145, 7105, 378, 283, 285, 777, 277, 367, 291, 225, 37, 1519, 9667, 827, 4806, 69, 7105, 471, 283, 375, 264, 269, 225, 51, 5278, 15742, 8522, 20190, 7105, 530, 283, 759, 264, 225, 44, 2949, 1076, 10984, 12568, 69, 17, 15401, 26147, 7105, 522, 283, 285, 264, 286, 225, 39, 952, 1767, 952, 1480, 15169, 4308, 1076, 7105, 595, 283, 368, 225, 49, 2079, 1943, 14808, 3987, 9402, 7105, 633, 283, 278, 334, 340, 264, 272, 225, 37, 86, 88, 4340, 1017, 84, 1480, 1206, 1865, 25496, 26017, 7105, 642, 283, 314, 264, 1044, 264, 225, 50, 93, 2769, 76, 7611, 69, 1074, 928, 1767, 9266, 7105, 663, 283, 316, 276, 299, 293, 264, 296, 225, 48, 1858, 73, 83, 3738, 76, 8046, 7105, 528, 283, 278, 264, 794, 264, 225, 39, 880, 6744, 8508, 6744, 7105, 705, 283, 281, 411, 304, 225, 52, 2360, 1678, 69, 

In [35]:
len(tokenized_squad_train_new)

2396

In [37]:
for i in tokenized_squad_val_new[:1]:
    print(i)
    

{'input_ids': [0, 317, 264, 343, 289, 275, 308, 264, 271, 289, 277, 309, 931, 264, 270, 264, 275, 293, 288, 286, 4112, 276, 272, 268, 316, 312, 333, 264, 270, 265, 4685, 2, 2, 317, 264, 343, 265, 266, 308, 264, 271, 289, 277, 309, 931, 264, 270, 264, 417, 288, 278, 265, 269, 267, 284, 267, 266, 265, 7217, 267, 273, 268, 861, 268, 1627, 267, 7218, 276, 270, 267, 271, 297, 289, 272, 9208, 4711, 288, 8148, 267, 266, 6, 472, 313, 265, 266, 276, 277, 1518, 281, 264, 284, 264, 321, 419, 276, 291, 442, 276, 317, 350, 274, 268, 270, 1059, 309, 271, 264, 270, 264, 266, 303, 267, 266, 268, 376, 267, 319, 2473, 267, 291, 268, 271, 931, 264, 270, 359, 4270, 281, 264, 272, 265, 266, 1030, 308, 276, 272, 264, 299, 3146, 268, 644, 265, 266, 384, 268, 497, 268, 273, 265, 395, 265, 377, 931, 264, 270, 264, 266, 475, 267, 543, 264, 269, 316, 350, 868, 268, 332, 264, 343, 497, 268, 266, 264, 291, 267, 277, 265, 266, 444, 264, 266, 268, 931, 264, 270, 264, 294, 268, 280, 265, 273, 265, 313, 352, 304, 289,

In [106]:
from transformers import TrainingArguments, Trainer, DefaultDataCollator

data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=44,
    per_device_eval_batch_size=16,
    num_train_epochs=300,
    weight_decay=0.01,
    eval_steps=1000,
    save_total_limit=1
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad_train_new,
    eval_dataset=tokenized_squad_val_new,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/user/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2402
  Num Epochs = 300
  Instantaneous batch size per device = 44
  Total train batch size (w. parallel, distributed & accumulation) = 44
  Gradient Accumulation steps = 1
  Total optimization steps = 16500


Step,Training Loss,Validation Loss
1000,0.020500,0.024596
2000,0.014800,0.009051
3000,0.013100,0.083566
4000,0.012000,0.088645
5000,0.011000,0.076335
6000,0.011100,0.106847
7000,0.010400,0.098923
8000,0.010600,0.095571
9000,0.009700,0.074631
10000,0.009700,0.068250


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Model weights saved in ./results/checkpoint-1500/pytorch_model.

Model weights saved in ./results/checkpoint-9500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-9500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-9500/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-9000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 262
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-10000
Configuration saved in ./results/checkpoint-10000/config.json
Model weights saved in ./results/checkpoint-10000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-10000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-10000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-9500] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-10500
Configuration saved in ./results/checkpoint-10500/config.json
Model weights saved in ./results/checkpoint-10500/pytorch_model.bin
tokenizer confi

TrainOutput(global_step=16500, training_loss=0.01164216694687352, metrics={'train_runtime': 5534.046, 'train_samples_per_second': 130.212, 'train_steps_per_second': 2.982, 'total_flos': 9.41486238830592e+16, 'train_loss': 0.01164216694687352, 'epoch': 300.0})

# Inference

In [40]:
from transformers import AutoModelForQuestionAnswering,AutoTokenizer,pipeline

model = AutoModelForQuestionAnswering.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer")
tokenizer = AutoTokenizer.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer")

for answer, context, question in zip(squad_test["answers"], squad_test["context"], squad_test["question"]):
    print("context:", context)
    print("question:", question)
    print("gt:", answer)
    QA = pipeline('question-answering', model=model, tokenizer=tokenizer)
    QA_input = {'question': question,'context':context}
    prediction = QA(QA_input)
    
    print(prediction)
    
    print("="*40)

context: ভারতের জাতীয় পতাকা হলো কেন্দ্রে চব্বিশটি দণ্ডযুক্ত নীল "অশোকচক্র" সহ গেরুয়া, সাদা ও সবুজ আনুভূমিক আয়তাকার ত্রিবর্ণরঞ্জিত পতাকা। ১৯৪৭ সালের ২২ জুলাই গণপরিষদের একটি অধিবেশনে এই পতাকার বর্তমান রূপটি ভারত অধিরাজ্যের সরকারি পতাকা হিসেবে গৃহীত হয়েছিল। পরবর্তীকালে এটি ভারতীয় প্রজাতন্ত্রের জাতীয় পতাকার মর্যাদা লাভ করে। ভারতে এই পতাকাটিকে সাধারণত "ত্রিরঙ্গা পতাকা" বা "ত্রিবর্ণরঞ্জিত পতাকা" বলা হয়। পিঙ্গালি ভেঙ্কাইয়া কৃত ভারতীয় জাতীয় কংগ্রেসের "স্বরাজ" পতাকার ভিত্তিতে এই পতাকাটির নকশা প্রস্তুত করা হয়েছিল।
question: ভারতীয় জাতীয় পতাকায় মোট কতগুলি রং থাকে ?
gt: {'answer_start': 109, 'text': 'ত্রিবর্ণ'}
{'score': 1.0, 'start': 109, 'end': 119, 'answer': 'ত্রিবর্ণরঞ'}
context:  সুন্নি মুসলিমরা ইসলাম অনুসারীদের মধ্যে সবচেয়ে বড় সম্প্রদায়। সুন্নিরা আরো পরিচিত আহলে সুন্নাত ওয়াল জামা'য়াত (Arabic: هل السنة والجماعة‎ ") সংক্ষেপে আহল আস-সুন্নাহ (Arabic: أهل السنة‎‎) নামে। সুন্নি শব্দের উৎপত্তি সুন্নাহ (Arabic: سنة‎) শব্দ থেকে যা দ্বারা ইস্লামের নবি মুহাম্মদের বাণী ও কর্মকে বুঝায়। 

{'score': 1.0, 'start': 172, 'end': 187, 'answer': 'বুকের ভিতর আগুন'}
context: সামবেদ (সংস্কৃত: सामवेद) (সামন্‌ বা গান ও বেদ বা জ্ঞান থেকে) হল সংগীত ও মন্ত্রের বেদ।[24] সামবেদ হিন্দুধর্মের সর্বপ্রধান ধর্মগ্রন্থ বেদের দ্বিতীয় ভাগ। এটি বৈদিক সংস্কৃত ভাষায় রচিত। সামবেদে ১,৮৭৫টি মন্ত্র রয়েছে।[25] এই শ্লোকগুলি মূলত বেদের প্রথম ভাগ ঋগ্বেদ থেকে গৃহীত।[26] এটি একটি প্রার্থনামূলক ধর্মগ্রন্থ। বর্তমানে সামবেদের তিনটি শাখার অস্তিত্ব রয়েছে। এই বেদের একাধিক পাণ্ডুলিপি ভারতের বিভিন্ন অঞ্চল থেকে আবিষ্কৃত হয়েছে।[27][28]
question: বেদের প্রথম ভাগ কী ?
gt: {'answer_start': 252, 'text': 'ঋগ্বেদ'}
{'score': 1.0, 'start': 252, 'end': 258, 'answer': 'ঋগ্বেদ'}
context: ১৯৩০ ফিফা বিশ্বকাপ ছিল প্রথম অনুষ্ঠিত কোন ফুটবল বিশ্বকাপ। জুলাই ১৩ থেকে জুলাই ৩০ পর্যন্ত এটি উরুগুয়েতে অনুষ্ঠিত হয়। ফিফা ১৯২৯ সালের বার্সেলোনা সেমিনারে উরুগুয়েকে বিশ্বকাপ আয়োজনের দায়িত্ব দেয় কেননা সেবছর উরুগুয়ের স্বাধীনতার শতবর্ষে পা দিয়েছিল এবং উরুগুয়ে ফুটবল দল সফল ভাবে ১৯২৮ গ্রীষ্ম অলিম্পিকে ফুটবল শিরোপা জিতেছিল।
question: সর্বপ

{'score': 0.004294564947485924, 'start': 794, 'end': 806, 'answer': 'ষষ্ঠ শতাব্দী'}
context: জার্মানি ইউরোপীয় ইউনিয়নের সবচেয়ে জনবহুল দেশ। সারা পৃথিবীতে জার্মানি ১৫তম জনবহুল দেশ।[11] প্রতি বর্গকিলোমিটারে ২২৯.৪ জন অধিবাসী বাস করে। জার্মান ভাষা জার্মানির সরকারি ভাষা। এছাড়া অভিবাসী জনগোষ্ঠীর মধ্যে আরও অনেকগুলি ভাষা প্রচলিত। এদের মধ্যে আরবি, গ্রিক, ইতালীয়, কুর্দি এবং তুর্কি ভাষা উল্লেখযোগ্য। লুসাটিয়া অঞ্চলে সর্বীয় ভাষা এবং দক্ষিণ শ্লেসভিগ অঞ্চলে ডেনীয় ভাষার আঞ্চলিক সরকারি মর্যাদা রয়েছে। উপকূলীয় দ্বীপগুলিতে ফ্রিজীয় ভাষা প্রচলিত। জার্মানির পূর্ব সীমান্তে পোলীয় ভাষা এবং উত্তর-পশ্চিম সীমান্তে ওলন্দাজ ভাষা প্রচলিত। এছাড়াও এখানে অনির্দিষ্ট সংখ্যক রোমানি ভাষাভাষী বাস করে।
question: জার্মানির সরকারি ভাষা কী ?
gt: {'answer_start': 139, 'text': 'জার্মান ভাষা'}
{'score': 1.0, 'start': 139, 'end': 151, 'answer': 'জার্মান ভাষা'}
context: “ভাগোয়া বা গেরুয়া রঙ ত্যাগ ও বৈরাগ্যের প্রতীক। আমাদের নেতৃবৃন্দকে পার্থিব লাভের প্রতি উদাসীন ও আপন আপন কাজে যত্নবান হইতে হইবে। মধ্যস্থলে সাদা আমাদের আত্ম

{'score': 1.0, 'start': 214, 'end': 218, 'answer': 'তিনভ'}
context: চৌধুরী রহমত আলি (উর্দু: چودھری رحمت علی) (১৬ নভেম্বর ১৮৯৫ – ১২ ফেব্রুয়ারি ১৯৫১) ছিলেন একজন পাকিস্তানি[1][2][3] মুসলিম জাতীয়তাবাদী। তিনি পাকিস্তান সৃষ্টির প্রবক্তাদের অন্যতম। দক্ষিণ এশিয়ায় মুসলিমদের পৃথক আবাসভূমির জন্য “পাকিস্তান” শব্দটি তার সৃষ্টি। তাকে পাকিস্তান প্রতিষ্ঠার আন্দোলনের স্থপতি হিসেবে তাকে বিবেচনা করা হয়। ১৯৩৩ সালে প্রকাশিত পুস্তিকা নাউ অর নেভার; আর উই টু লিভ অর পেরিশ ফরএভার? এর লেখক হিসেবে তিনি খ্যাত। এটি পাকিস্তান ঘোষণা বলেও পরিচিত। পুস্তিকাটি নিম্নোক্তভাবে শুরু হয়ঃ
question: পাকিস্তানি মুসলিম জাতীয়তাবাদী চৌধুরী রহমত আলি কবে জন্মগ্রহণ করেন ?
gt: {'answer_start': 42, 'text': '১৬ নভেম্বর ১৮৯৫'}
{'score': 1.0, 'start': 42, 'end': 57, 'answer': '১৬ নভেম্বর ১৮৯৫'}
context:   গণপ্রজাতন্ত্রী বাংলাদেশের সংবিধান স্বাধীন ও সার্বভৌম বাংলাদেশ রাষ্ট্রের সর্বোচ্চ আইন। এটি একটি লিখিত দলিল। ১৯৭২ খ্রিস্টাব্দের ৪ঠা নভেম্বর তারিখে বাংলাদেশের জাতীয় সংসদে এই সংবিধান গৃহীত হয় এবং একই বছরের ১৬ই ডিসেম্বর অর্থাৎ বাংলাদে

{'score': 0.9999781847000122, 'start': 360, 'end': 375, 'answer': 'পশ্চিম গোলার্ধে'}
context: ১৮৯৭ খ্রিস্টাব্দের ২৩ জানুয়ারি, বর্তমান ওডিশা রাজ্যের কটক শহরে (ওডিয়া বাজার) জন্মগ্রহণ করেন সুভাষচন্দ্র বসু। তিনি ছিলেন কটক-প্রবাসী বিশিষ্ট বাঙালি আইনজীবী জানকীনাথ বসু ও প্রভাবতী দেবীর চৌদ্দ  সন্তানের মধ্যে নবম। ষষ্ঠ শ্রেণি পর্যন্ত সুভাষচন্দ্র একটি কটকের ইংরেজি স্কুলে পড়াশোনা করেন; বর্তমানে এই স্কুলটির নাম স্টুয়ার্ট স্কুল। এরপর তাঁকে ভর্তি করা হয় কটকের রাভেনশ কলেজিয়েট স্কুলে। সুভাষচন্দ্র ছিলেন মেধাবী ছাত্র। ১৯১১ খ্রিস্টাব্দে ম্যাট্রিকুলেশন পরীক্ষায় কলকাতা থেকে প্রথম স্থান অধিকার করেন তিনি। ১৯১৮ খ্রিস্টাব্দে কলকাতা বিশ্ববিদ্যালয় অধিভুক্ত স্কটিশ চার্চ কলেজ থেকে দর্শনে সাম্মানিকসহ বিএ পরীক্ষায় উত্তীর্ণ হন।
question: নেতাজী সুভাষ চন্দ্র বসু কোন শহরে জন্মগ্রহণ করেন ?
gt: {'answer_start': 55, 'text': 'কটক'}
{'score': 1.0, 'start': 55, 'end': 58, 'answer': 'কটক'}
context: নেইমার ডা সিল্ভা জন্মগ্রহণ করেন সাও পাওলো, ব্রাজিলে সিনিয়র নেইমার ডা সিল্ভা এবং নান্দিনি সান্তসের ঘরে। তিনি তাঁর পিতার ন

...

{'score': 1.0, 'start': 386, 'end': 399, 'answer': 'চতুর্থ ইন্দ্র'}
context: খোট্টিগ অমোঘবর্ষের রাজত্বকালে পারমার রাজা সিকায় হর্ষ রাষ্ট্রকূট সাম্রাজ্য আক্রমণ করে মান্যখেত লুণ্ঠন করেন। এর ফলে রাষ্ট্রকূটদের শক্তি হ্রাস পায়। এরপরই রাষ্ট্রকূট সাম্রাজ্যের পতন ঘটে।[73] এই পরাজয়ের সুযোগ নিয়ে তারদাবাদি প্রদেশের (আধুনিক বিজাপুর জেলা, কর্ণাটক) শাসক তৃতীয় তৈলপ নিজেকে স্বাধীন রাজা ঘোষণা করলে রাষ্ট্রকূট সাম্রাজ্যের পতন সম্পূর্ণ হয়।[74][75] সর্বশেষ রাষ্ট্রকূট সম্রাট চতুর্থ ইন্দ্র জৈন সন্ন্যাসীদের প্রথা অনুসারে শ্রবণবেলগোলায় অনশনে মৃত্যুবরণ করেন। রাষ্ট্রকূটদের পতনের সঙ্গে সঙ্গে দাক্ষিণাত্য ও উত্তর ভারতে তাদের সামন্ত শাসকেরা স্বাধীনতা ঘোষণা করেন। পশ্চিম চালুক্য সাম্রাজ্য ১০১৫ খ্রিস্টাব্দে মান্যখেত দখল করে সেখানেই তাদের রাজধানী স্থাপন করে। এরপর একাদশ শতাব্দীতে মান্যখেত পশ্চিম চালুক্যদের সাম্রাজ্যের কেন্দ্রে পরিণত হয়। শাসনকেন্দ্র কৃষ্ণা নদী ও গোদাবরী নদীর দোয়াব বেঙ্গিতে স্থানান্তরিত হয়। পশ্চিম দাক্ষিণাত্যে রাষ্ট্রকূটদের পূর্বতন সামন্ত শাসকদের চালুক্যরা নিজেদের অধীনে নিয়ে আসে। এরপর তাঁরা দাক্ষ

{'score': 1.0, 'start': 985, 'end': 989, 'answer': '২০১৮'}
context: সাধু মথি লিখিত সুসমাচার ও লুক এর সুসমাচার এ মেরি একজন কুমারীরূপে বর্ণিত হয়েছেন।[4]  খ্রিস্ট ধর্মাবলম্বীরা বিশ্বাস করেন যে মেরি পবিত্র আধ্যাত্মিক শক্তি দ্বারা কুমারী অবস্থাতেই সন্তানের জন্ম দেন। এই অলৌকিক জন্ম যখন ঘটে তখন মেরি যোসেফ এর বাগদত্তা ছিলেন এবং বিবাহের আনুষ্ঠানিক গৃহপ্রবেশের জন্য অপেক্ষা করছিলেন। যোসেফকে বিয়ে করে মেরি বেথেলহাম চলে আসেন যেখানে যিশুর জন্ম হয়।[5][6] লুক এর সুসমাচার এ বলা হয়েছে দেবদূত জিবরাঈল মেরির সম্মুখে আবির্ভূত হন এবং যিশুর মা হওয়ার জন্য মেরি দৈবভাবে নির্বাচিত হয়েছেন বলে জানান।  ক্যানোনিক্যাল সুসমাচার অনুযায়ী মেরি যিশুর ক্রুশবিদ্ধকরণ এর সময় উপস্থিত ছিলেন এবং জেরুসালেম এর প্রাথমিক খ্রিস্টান সম্প্রদায়ের সদস্য ছিলেন।ক্যাথলিক ও অর্থোডক্স ভাষ্যমতে জীবনাবসানের পর মেরির দেহ সরাসরি স্বর্গে নিয়ে যাওয়া হয় যা ‘এসাম্পশন অফ মেরি’ নামে পরিচিত।[7][8]
question: যিশু খ্রিস্টের বাবার নাম কী ?
gt: {'answer_start': 227, 'text': 'যোসেফ'}
{'score': 1.0, 'start': 227, 'end': 232, 'answer': 'যোসেফ'}
conte

{'score': 0.8345754742622375, 'start': 658, 'end': 670, 'answer': 'কর্নেল আবদুর'}
context: মিয়ানমার বা মায়ানমার (Burmese: မြန်မာ মিয়ামা, আ-ধ্ব-ব: [mjəmà]); প্রাক্তন নাম ও কথ্যরূপ বর্মা বা বার্মা (Burmese: ဗမာ বামা, আ-ধ্ব-ব: [bəmà]); প্রাচীন নাম ব্রহ্মদেশ) দক্ষিণ-পূর্ব এশিয়ার একটি রাষ্ট্র। মায়ানমারের আনুষ্ঠানিক নাম হল মায়ানমার প্রজাতন্ত্র (Burmese: ပြည်ထောင်စု သမ္မတ မြန်မာ နိုင်ငံတော် [pjìdàʊɴzṵ θàɴməda̰ mjəmà nàɪɴŋàɴdɔ̀] পিডাওঁযু থাঁমাডা মিয়ামা নাইঁঙাঁড)। মায়ানমারের রাজধানী নেপিডো (နေပြည်တော် [nèpjìdɔ̀] নেপিড)। তৎকালীন বার্মার গণতান্ত্রিক সরকারের উৎখাতের পর ১৯৮৯ সালে সেখানকার সামরিক সরকার বার্মার নতুন নামকরণ করে "মিয়ানমার" এবং প্রধান শহর ও তৎকালীন রাজধানী রেঙ্গুনের নতুন নাম হয় "ইয়াঙ্গুন"। তবে গণতান্ত্রিক দলগুলোর অনেক অনুসারীই এই নামকরণের বিপক্ষে। ২১ অক্টোবর ২০১০ থেকে দেশটির জাতীয় সঙ্গীত ও নতুন জাতীয় পতাকা প্রবর্তন করা হয়।
question: মায়ানমারের রাজধানী কোনটি ?
gt: {'answer_start': 396, 'text': 'নেপিডো'}
{'score': 0.991908848285675, 'start': 157, 'end': 166, 'answer': 'ব্রহ

{'score': 1.0, 'start': 33, 'end': 47, 'answer': 'রাঘবেন্দ্র রাও'}
context: বাংলাদেশের রাজধানী ঢাকাতে নির্মিতব্য ভূ-উপরিস্থ রেল ব্যবস্থার নাম ঢাকা মেট্রো। ঢাকা মেট্রো রেল ব্যবস্থাকে কে ম্যাস র‍্যাপিড ট্রানযিট সংক্ষেপে এমআরটি (MRT) হিসেবেও উল্লেখ করা হয়। ২০০০ সাল থেকে অতি জনবহুল ঢাকা মহানগরীর ক্রমঃবর্ধমান যানবাহন সমস্যা ও পথের দু:সহ যানজট কমিয়ে আনার লক্ষ্যে এই ম্যাস র‍্যাপিড ট্রানযিট ব্যবস্থা স্থাপনের পরিকল্পনা করা হয়েছে।  এই উদ্দেশ্যে একটি উন্নয়ন প্রকল্প গ্রহণ করা হয় যার নাম ঢাকা মাস র‌্যাপিড ট্রানজিট ডেভেলপমেন্ট প্রজেক্ট। ২০১৬ থেকে এই পরিকল্পনার বাস্তবায়ন শুরু হয়েছে। ২০১৯ সালের ডিসেম্বরের মধ্যে এর বাণিজ্যিক পরিচালনা শুরুর লক্ষ্য স্থির করা হয়েছে। এই প্রকল্প বাস্তবায়নে ব্যয় হবে ২২ হাজার কোটি টাকা যার মধ্যে জাপান ইন্টারন্যাশনাল কো-অপারেশন এজেন্সি (জাইকা)’র প্রকল্প ঋণ ৭৫ শতাংশ এবং বাংলাদেশ সরকারের নিজস্ব অর্থায়ন ২৫ শতাংশ।[4]
question: ঢাকা মেট্রো নির্মাণের কাজ কবে শুরু হয় ?
gt: {'answer_start': 458, 'text': '২০১৬'}
{'score': 0.9999996423721313, 'start': 458, 'end': 462, 'answer

{'score': 1.0, 'start': 1120, 'end': 1124, 'answer': '১৯৭৫'}


# Single Inference

In [4]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline



model = AutoModelForQuestionAnswering.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer")
tokenizer = AutoTokenizer.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer")

context = "বাংলাদেশ ও ভারতের অনেক বৃহৎ নদী পূর্ব থেকে পশ্চিমে প্রবাহিত হয়ে বঙ্গোপসাগরে পতিত হয়েছে।\
তন্মধ্যে উত্তরদিক থেকে গঙ্গা, মেঘনা এবং ব্রহ্মপুত্র; দক্ষিণদিক থেকে মহানদী, গোদাবরী, কৃষ্ণা, ইরাবতী এবং কাবেরী নদী উল্লেখযোগ্য।\
৬৪ কিলোমিটারব্যাপী (৪০ মাইল) কৌম নদী সবচেয়ে ছোট নদী হিসেবে সরু খাল দিয়ে এবং ২,৯৪৮ কিলোমিটারব্যাপী (১,৮৩২ মাইল)\
বিশ্বের ২৮তম দীর্ঘ নদী হিসেবে ব্রহ্মপুত্র নদ বাংলাদেশ, চীন, নেপাল ও ভারতের মধ্য দিয়ে প্রবাহিত হয়ে বঙ্গোপসাগরে মিলিত হয়েছে।\
সুন্দরবন ম্যানগ্রোভ বনাঞ্চল গঙ্গা, ব্রহ্মপুত্র ও মেঘনা নদীর ব-দ্বীপকে ঘিরে গঠিত হয়েছে। মায়ানমারের (সাবেক বার্মা) ইরাওয়াদি (সংস্কৃত ইরাবতী)\
নদীও এ উপসাগরে মিলিত হয়েছে এবং একসময় গভীর ও ঘন ম্যানগ্রোভ বনাঞ্চলের সৃষ্টি করেছিল।"
 
question = "ব্রহ্মপুত্র নদের মোট দৈর্ঘ্য কত ?"

QA = pipeline('question-answering', model=model, tokenizer=tokenizer)
QA_input = {'question': question,'context':context}

prediction = QA(QA_input)
print(prediction)

{'score': 1.0, 'start': 294, 'end': 310, 'answer': '২,৯৪৮ কিলোমিটারব'}


In [24]:
import torch
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



context = "বাংলাদেশ ও ভারতের অনেক বৃহৎ নদী পূর্ব থেকে পশ্চিমে প্রবাহিত হয়ে বঙ্গোপসাগরে পতিত হয়েছে।\
তন্মধ্যে উত্তরদিক থেকে গঙ্গা, মেঘনা এবং ব্রহ্মপুত্র; দক্ষিণদিক থেকে মহানদী, গোদাবরী, কৃষ্ণা, ইরাবতী এবং কাবেরী নদী উল্লেখযোগ্য।\
৬৪ কিলোমিটারব্যাপী (৪০ মাইল) কৌম নদী সবচেয়ে ছোট নদী হিসেবে সরু খাল দিয়ে এবং ২,৯৪৮ কিলোমিটারব্যাপী (১,৮৩২ মাইল)\
বিশ্বের ২৮তম দীর্ঘ নদী হিসেবে ব্রহ্মপুত্র নদ বাংলাদেশ, চীন, নেপাল ও ভারতের মধ্য দিয়ে প্রবাহিত হয়ে বঙ্গোপসাগরে মিলিত হয়েছে।\
সুন্দরবন ম্যানগ্রোভ বনাঞ্চল গঙ্গা, ব্রহ্মপুত্র ও মেঘনা নদীর ব-দ্বীপকে ঘিরে গঠিত হয়েছে। মায়ানমারের (সাবেক বার্মা) ইরাওয়াদি (সংস্কৃত ইরাবতী)\
নদীও এ উপসাগরে মিলিত হয়েছে এবং একসময় গভীর ও ঘন ম্যানগ্রোভ বনাঞ্চলের সৃষ্টি করেছিল।"
 
question = "ব্রহ্মপুত্র নদের মোট দৈর্ঘ্য কত ?"

model = AutoModelForQuestionAnswering.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer").to(device)
tokenizer = AutoTokenizer.from_pretrained("saiful9379/Bangla_Roberta_Question_and_Answer")


input_ids = tokenizer.encode(question, context)
tokens = tokenizer.convert_ids_to_tokens(input_ids)

output = model(torch.tensor([input_ids]).to(device))

argmax_start = torch.argmax(output['start_logits']).to(device)
argmax_end = torch.argmax(output['end_logits']).to(device)
print("="*40)
print("Question: ", question)
print("Context:", context)
print("-"*40)
print("Predicted answer: ", "".join(tokens[argmax_start:argmax_end]))

predicted_string = "".join(tokens[argmax_start:argmax_end])

token_id = tokenizer.encode(predicted_string)
print(argmax_start, argmax_end)
answer_tokens = tokens[argmax_start: argmax_end+1]

answer = tokenizer.decode(tokenizer.convert_tokens_to_ids(answer_tokens))

print("answer : ", answer)

# random_data = jaquad_data["validation"][random.randint(0, len(jaquad_data["validation"]))]
# from_data_to_answer(model, tokenizer, random_data)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-16.3680, -17.3164, -17.7502, -18.2334, -17.5051, -17.6700, -18.8044,
         -17.0317, -17.5096, -19.1373, -18.0707, -18.0804, -18.6880, -17.8809,
         -18.5109, -18.2540, -18.3011, -17.8451, -18.6201, -17.4243, -19.2553,
         -17.7516, -18.9651, -18.4140, -18.7526, -18.6903, -18.5156, -16.8956,
         -17.9776, -15.9104, -17.5950, -18.4859, -18.6443, -19.7269, -18.9075,
         -18.8117, -17.3282, -18.9628, -18.9563, -19.4569, -15.8712, -18.5416,
         -18.6439, -14.6385, -17.8338, -16.2377, -17.7837, -18.2652, -17.1790,
         -18.0542, -17.8363, -16.9750, -17.9468, -17.1633, -18.5397, -17.7660,
         -18.7768, -17.0808, -17.3248, -16.9080, -17.6782, -17.6809, -19.8366,
         -18.1068, -17.5276, -16.2343, -17.6140, -15.3399, -17.5103, -16.3853,
         -15.9636, -16.8899, -17.3155, -17.4412, -16.4900, -18.3246, -17.2063,
         -17.6613, -18.0371, -19.1178, -17.3419, -18.0175, -15.7748, -16.0610,